In [1]:
import re
import numpy as np
import pandas as pd

path = 'E:/BaiduYunDownload/DC_2/'

borrow_train = pd.read_csv(path+'train/borrow_train.txt',sep = '\n',header = None,names = ['col'])
borrow_train = borrow_train.drop_duplicates()

borrow_test = pd.read_csv(path+'test/borrow_test.txt',sep = '\n',header = None,names = ['col'])
borrow_test = borrow_test.drop_duplicates()

borrow = pd.concat([borrow_train,borrow_test])

borrow['student_id'] = borrow['col'].apply(lambda x:x.split(',')[0])
borrow['borrow_time'] = borrow['col'].apply(lambda x:x.split(',')[1])
borrow['book'] = borrow['col'].apply(lambda x:''.join([i.split(" ")[0]for i in x.split(',')[2:-1]]))
borrow['book_num'] =  borrow['col'].apply(lambda x:x.split(',')[-1] if x.split(',')[-1].startswith('"') else 'NNN')
borrow['book_num'] = borrow['book_num'].str.replace('"','')
borrow['borrow_time'] = borrow['borrow_time'].str.replace('"','')
borrow['book'] = borrow['book'].str.replace('"','')

pattern = re.compile(r'[A-Za-z]')
def book_tag(x):
    if x.startswith('TP') or x.startswith('TN') or x.startswith('TM') or x.startswith('TB') or x.startswith('TH'):
        return x[0:2]
    elif x!='NNN' and re.match(pattern,x):
        return x[0]
    elif x=='NNN' or not re.match(pattern,x):
        return x
borrow['book_num1'] = borrow['book_num'].apply(lambda x: book_tag(x))

borrow1 = borrow.groupby('book_num1').filter(lambda x:len(x)>=10)
borrow2 = borrow.groupby('book_num1').filter(lambda x:len(x)<10)
borrow2['book_num1'] = 'NNN'
borrow = pd.concat([borrow1,borrow2])

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [2]:
######na填充
book = borrow[borrow['book_num1']!='NNN'][['book','book_num1']].drop_duplicates()
book = book.rename(columns={'book_num1':'book_num2'})
borrow1 = pd.merge(borrow,book,on = 'book',how = 'left')
borrow1 = borrow1.fillna('NNN')
borrow2 = borrow1[(borrow1['book_num1']=='NNN')]
borrow3 = borrow1[(borrow1['book_num1']!='NNN')]
borrow2['book_num1'] = borrow2['book_num2']
borrow4 = pd.concat([borrow2,borrow3])
borrow4 = borrow4.drop(['col','book_num','book_num2'],axis = 1)

borrow4['borrow_time'] = pd.to_datetime(pd.Series(borrow4['borrow_time']))

borrow4['weekday'] = [borrow4['borrow_time'][i].dayofweek for i in range(len(borrow4['borrow_time']))]
borrow4['month'] = borrow4['borrow_time'].apply(lambda x:str(x)[5:7])

borrow5 = borrow4.groupby(['student_id','book_num1'])['book'].count()
borrow5 = borrow5.unstack(fill_value=0) 
borrow5 = borrow5.reset_index()

borrow6 = borrow4.groupby(['student_id'],as_index = False)['book'].count()
borrow6 = borrow6.rename(columns = {'book':'book_count'})

D:\Anaconda2\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
dorm_train = pd.read_csv(path+'train/dorm_train.txt',header = None)
dorm_train.columns = ['student_id','dorm_time','out1_in0']
dorm_train = dorm_train.drop_duplicates()

dorm_test = pd.read_csv(path+'test/dorm_test.txt',header = None)
dorm_test.columns = ['student_id','dorm_time','out1_in0']
dorm_test = dorm_test.drop_duplicates()

dorm = pd.concat([dorm_train,dorm_test])

dorm['hour']=dorm['dorm_time'].apply(lambda x:str(x)[11:13])
dorm['month']=dorm['dorm_time'].apply(lambda x:str(x)[5:7])
dorm_in_count = dorm.groupby(['student_id','hour'])['out1_in0'].count()
dorm_in_count = dorm_in_count.unstack(fill_value=0)
dorm_in_count = dorm_in_count.reset_index()

dorm_out_count = dorm[dorm['out1_in0']==1].groupby(['student_id','hour'])['out1_in0'].count()
dorm_out_count = dorm_out_count.unstack(fill_value=0)
dorm_out_count = dorm_out_count.reset_index()
dorm_count = dorm.groupby(['student_id'],as_index = False)['dorm_time'].count()
dorm_count = dorm_count.rename(columns = {'dorm_time':'dorm_count'})

In [4]:
library_train = pd.read_csv(path+'train/library_train.txt',header = None)
library_train.columns = ['student_id','library_door','library_time']
library_train = library_train.drop_duplicates()

library_test = pd.read_csv(path+'test/library_test.txt',header = None)
library_test.columns = ['student_id','library_door','library_time']
library_test = library_test.drop_duplicates()

library = pd.concat([library_train,library_test])
library['hour']=library['library_time'].apply(lambda x:str(x)[11:13])
library['month']=library['library_time'].apply(lambda x:str(x)[5:7])

library_door = library.groupby(['student_id','library_door'])['library_time'].count()
library_door = library_door.unstack(fill_value=0)
library_door = library_door.reset_index()

library_count = library.groupby(['student_id'],as_index = False)['library_time'].count()
library_count = library_count.rename(columns = {'library_time':'library_count'})

In [5]:
score_train = pd.read_csv(path+'train/score_train.txt',header = None)
score_train.columns = ['student_id','college','class_rank']
score_train = score_train.drop_duplicates()

score_test = pd.read_csv(path+'test/score_test.txt',header = None)
score_test.columns = ['student_id','college','class_rank']
score_test = score_test.drop_duplicates()

score = pd.concat([score_train,score_test])
college1 = score.groupby(['college'],as_index = False)['class_rank'].max()
college2 = score.groupby(['college'],as_index = False)['class_rank'].min()
college1 = college1.rename(columns = {'class_rank':'class_rank_max'})
college2 = college2.rename(columns = {'class_rank':'class_rank_min'})
score_train_test1 = pd.merge(score, college1, how='left',on='college')
score_train_test = pd.merge(score_train_test1, college2, how='left',on='college')
score_train_test['order1'] = (score_train_test['class_rank']-score_train_test['class_rank_min'])/(score_train_test['class_rank_max']-
                                                                                                  score_train_test['class_rank_min'])
#score_train_test['order2'] = score_train_test['order1']/score_train_test['college']
score_train_test = score_train_test.drop(['class_rank_max','class_rank_min'],axis = 1)

subsidy_train = pd.read_csv(path+'train/subsidy_train.txt',header = None)
subsidy_train.columns = ['student_id','grant_amount']

subsidy_test = pd.read_csv(path+'test/subsidy_test.txt',header = None)
subsidy_test.columns = ['student_id']
subsidy_test['grant_amount'] = 0
subsidy_test = subsidy_test.drop_duplicates()

subsidy_train['TT'] = 'train'
subsidy_test['TT'] = 'test'
subsidy = pd.concat([subsidy_train,subsidy_test])

In [6]:
card_data1 = pd.read_csv(path+'card_data1.0219.csv',encoding = 'gb18030',index_col=None)
card_data2 = pd.read_csv(path+'card_data2.0219.csv',encoding = 'gb18030',index_col=None)

In [7]:
data1 = pd.merge(subsidy,score_train_test,on = 'student_id',how = 'outer')
data2 = data1.merge(library_count,on = 'student_id',how = 'outer')
data3 = data2.merge(library_door,on = 'student_id',how = 'outer')
data4 = data3.merge(dorm_in_count,on = 'student_id',how = 'outer')
data5 = data4.merge(dorm_out_count,on = 'student_id',how = 'outer')
data6 = data5.merge(dorm_count,on = 'student_id',how = 'outer')
data7 = data6.merge(borrow5,on = 'student_id',how = 'outer')
data8 = data7.merge(borrow6,on = 'student_id',how = 'outer')
data9 = data8.merge(card_data1,on = 'student_id',how = 'outer')
data10 = data9.merge(card_data2,on = 'student_id',how = 'outer')
#data9 = data8.merge(card_data3,on = 'student_id',how = 'outer')
#data10 = data9.merge(card_data4,on = 'student_id',how = 'outer')

D:\Anaconda2\lib\site-packages\pandas\indexes\base.py:2097: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  indexer = self._engine.get_indexer(target._values)
D:\Anaconda2\lib\site-packages\pandas\indexes\base.py:1237: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return key in self._engine
D:\Anaconda2\lib\site-packages\pandas\indexes\base.py:941: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  return self._engine.is_unique


In [8]:
data10.fillna(-1).to_csv(path+'data0219.csv',encoding = 'gb18030',index = False)